In [1]:
import tensorflow as tf

C:\Users\KSHITIJ ANAND\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\KSHITIJ ANAND\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\KSHITIJ ANAND\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\KSHITIJ ANAND\anaconda3\lib\site-packages\tensorflow\python\framework\dty

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist  = input_data.read_data_sets("MNIST_data/" ,one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
input_width = 28
input_height = 28
input_channels = 1
n_pixels = 784

n_conv1 = 32
n_conv2 = 64

conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2
n_out = 10
n_hidden = 1024

stride_size_conv1 = 1
stride_size_conv2 = 1

input_size_hidden = (input_height//(max_pool1_k*max_pool2_k))*(input_width//(max_pool1_k*max_pool2_k))*n_conv2

In [4]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_hidden , n_hidden])),  
    "wo" : tf.Variable(tf.random_normal([n_hidden,n_out]))
    
}

bias = {
    
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out])),
    
}

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
def conv(x,weights,bias, strides = 1):
    #moving filter using inbuilt function
    out = tf.nn.conv2d(x, weights , padding = "SAME" , strides = [1,strides,strides,1])
    
    #now add bias to it
    
    out = tf.nn.bias_add(out,bias)
    #pass it through actvation func
    
    out = tf.nn.relu(out)
    
    return out
    
    
def max_pool(x , k=2):
     
    return tf.nn.max_pool(x , padding = "SAME" , ksize = [1,k,k,1] , strides = [1,k,k,1])#stride same as ksize

In [8]:
def cnn(x,weights,bias , keep_prob):
    #first reshape into the normal 2d form and considering the no. of i/p as well
    
    x = tf.reshape(x , shape = [-1,input_height,input_width,input_channels])
    
    #now pass it through first cnn layer
    
    conv1 = conv(x,weights["wc1"],bias["bc1"],stride_size_conv1)
    conv1_pool = max_pool(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights["wc2"],bias["bc2"] , stride_size_conv2)
    conv2_pool = max_pool(conv2, max_pool2_k)
    
    #now we need to pass through hidden layers
    
    #first reshape it
    hidden_input = tf.reshape(conv2_pool , shape = [-1, input_size_hidden])
    
    #now pass it through dense layer
    
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights["wh1"]) , bias["bh1"])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    
    hidden_output = tf.nn.dropout(hidden_output_before_dropout , keep_prob)
    
    output = tf.add(tf.matmul(hidden_output , weights["wo"]) , bias["bo"])
    
    return output

In [9]:
x = tf.placeholder("float", [None,n_pixels])
y = tf.placeholder(tf.int32,[None,n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x,weights,bias ,keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
#cross entropy function(inbuilt)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred,labels = y))

In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.02)
optimize = optimizer.minimize(cost)

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(batch_size):
        batch_x , batch_y = mnist.train.next_batch(batch_size)
        c , u = sess.run([cost,optimize] , feed_dict = {x:batch_x, y: batch_y , keep_prob : 0.8})
        total_cost+=c
        
    print(total_cost)

698952.70997715
1196.3193697929382
636.466108083725
322.26271986961365
259.76810002326965
264.74274230003357
246.40481662750244
244.11212515830994
242.2299382686615
235.81641960144043
236.43429827690125
234.3372859954834
235.48924493789673
232.870614528656
231.44672751426697
233.0394365787506
231.96127605438232
232.5563952922821
231.07784605026245
231.44651103019714
230.45573830604553
231.43058061599731
232.75725531578064
230.17845821380615
230.37295627593994


In [ ]:
predictions = tf.argmax(pred,1)
correct_labels = tf.argmax(y,1)
correct_prediction = tf.equal(predictions,correct_labels)
tr,df,er = sess.run([predictions,correct_labels,correct_prediction] , feed_dict = {x:mnist.test.images , y: mnist.test.labels , keep_prob : 1.0})
er.sum()